In [2]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
import os
import pandas as pd
import json
from sentence_transformers import SentenceTransformer

In [4]:
# Загружаем модель для эмбеддингов
model = SentenceTransformer('ai-forever/ru-en-RoSBERTa')

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Загружаем данные из файла
with open('./characteristics/time_series.json', 'r', encoding='utf-8') as f:
    ts_data = json.load(f)

In [6]:
# Подгружаем характеристики
ts_characteristics = ts_data['sets']
ts_characteristics_extended = {k['name']: {'description': k['description']} for k in ts_characteristics}

In [8]:
# Расчет дополнительных признаков
for k in ts_characteristics_extended.keys():
    ts = pd.read_csv(f'./datasets/{k}.csv')

    # Число наблюдений
    ts_characteristics_extended[k]['n_obs'] = ts['date'].nunique()

    # Периодичность наблюдений в секундах
    date_series = pd.to_datetime(ts['date'].drop_duplicates())
    date_series = date_series.sort_values()
    date_diffs = date_series.diff().dropna().dt.total_seconds()
    ts_characteristics_extended[k]['periodicity'] = date_diffs.mean()

    # Провекта на регулярность
    median = date_diffs.median()
    regular_fraction = (date_diffs == median).mean()
    ts_characteristics_extended[k]['regular_fraction'] = regular_fraction == 1.0


In [25]:
# Подгрузка TFB признаков
for k in ts_characteristics_extended:
    df_characteristics = pd.read_csv(f'./characteristics/TFB_characteristics_{k}.csv')
    tfb_characteristics = df_characteristics.drop(columns='Variable').abs().mean()
    ts_characteristics_extended[k].update(tfb_characteristics.to_dict())

In [27]:
ts_characteristics_extended['PEMS-BAY']

{'description': 'PEMS-BAY — датасет дорожного трафика района залива Сан-Франциско (Bay Area, Калифорния). Содержит измерения средней скорости движения транспорта (миль/час) по дорожным сенсорам. Каждый канал - один сенсор. Данные собраны с января по май 2017 года. Целевая переменная – средняя скорость движения по каждому сенсору. Экзогенные переменные отсутствуют. Рядые вещественные, регулярные. Категориальные данные отсутствуют. Наблюдается выраженная суточная и недельная сезонность.',
 'n_obs': 52116,
 'periodicity': 300.0,
 'regular_fraction': True,
 'Transition': 0.00502447915321245,
 'Shifting': 0.09043013514231561,
 'Seasonality': 0.6293320960104397,
 'Trend': 0.14460864559707,
 'Stationarity': 7.554315902209213e-28,
 'Correlation': 0.8424549153995613}